In [1]:
import os
import numpy as np
import scipy
import pandas as pd
import math 
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [2]:
vente_2015 = pd.read_csv("C:/Users/silva/PycharmProjects/TestExtract/FichierMock_réduit_100K.csv", sep = r'\s*;\s*', header = 0, encoding ="latin-1", engine = 'python')

#Attribution d'une note 1 vente = 1pts
vente_2015['note'] = 1

#Pour la suite du notebook, on va se concentrer sur deux catégories
vente_2015_autre = vente_2015[(vente_2015['categorie_produit'] == 'Autre') | (vente_2015['categorie_client'] == 'Autre')]

vente_2015_high_tech = vente_2015[(vente_2015['categorie_produit'] == 'High-tech') | (vente_2015['categorie_client'] == 'High-tech')]

print('Nombre de ventes par un client ou dun produit "Autre" : %d' % len(vente_2015_autre))

print('Nombre de ventes par un client ou dun produit "High-tech" : %d' % len(vente_2015_high_tech))


Nombre de ventes par un client ou dun produit "Autre" : 346
Nombre de ventes par un client ou dun produit "High-tech" : 19575


In [3]:
#On regroupe d'abord les ventes par client/produit ... 
vente_2015_unique_autre = vente_2015_autre.groupby(['id_client', 'id_produit']).size().groupby('id_client').size()
print("Nombre de client pour catégorie Autre %d" % len(vente_2015_unique_autre))

vente_2015_unique_high_tech = vente_2015_high_tech.groupby(['id_client', 'id_produit']).size().groupby('id_client').size()
print("Nombre de client pour catégorie High-tech %d" % len(vente_2015_unique_high_tech))

#... pour ensuite évaluer le nombre d'achat unique il a fait
#Nous gardons seulement les clients qui ont plus de 2 achats
#Cela pour eviter les problèmes de cold-start
vente_2015_autre_suffisant = vente_2015_unique_autre[vente_2015_unique_autre >= 2].reset_index()[['id_client']]
print("Nombre de client ayant fait un nombre suffisant (2) d'achat Autre %d" % len(vente_2015_autre_suffisant))

vente_2015_high_tech_suffisant = vente_2015_unique_high_tech[vente_2015_unique_high_tech >= 2].reset_index()[['id_client']]
print("Nombre de client ayant fait un nombre suffisant (2) d'achat high_tech %d" % len(vente_2015_high_tech_suffisant))

Nombre de client pour catégorie Autre 194
Nombre de client pour catégorie High-tech 5573
Nombre de client ayant fait un nombre suffisant (2) d'achat Autre 49
Nombre de client ayant fait un nombre suffisant (2) d'achat high_tech 2988


In [4]:
#On join le fichier de base avec les données que nous avons récupéré (les clients ayant acheté un nombre suffisant de produit)
#Afin d'avoir les informations de la vente initiale
vente_2015_entier_autre = vente_2015.merge(vente_2015_autre_suffisant, 
                                how = 'right',
                                left_on = 'id_client',
                                right_on = 'id_client')
print("Nombre de vente autre  %d" % len(vente_2015_entier_autre))

vente_2015_entier_high_tech = vente_2015.merge(vente_2015_high_tech_suffisant, 
                                how = 'right',
                                left_on = 'id_client',
                                right_on = 'id_client')
print("Nombre de vente high-tech  %d" % len(vente_2015_entier_high_tech))

Nombre de vente autre  435
Nombre de vente high-tech  23445


In [5]:
#Pour chaque interaction, nous allons ajouter un facteur, ce qui montre qu'il est est très intéressé par ce produit
def smooth_user_preference(x):
    return math.log(1+x, 2)

#On aggrege alors sur la note et on enlève les doublons
vente_autre = vente_2015_entier_autre.groupby(['id_client', 'id_produit'])['note'].sum().apply(smooth_user_preference).reset_index()
print("Nombre de vente unique client/produit autre %d" % len(vente_autre))

vente_high_tech = vente_2015_entier_high_tech.groupby(['id_client', 'id_produit'])['note'].sum().apply(smooth_user_preference).reset_index()
print("Nombre de vente unique client/produit high-tech %d" % len(vente_high_tech))

Nombre de vente unique client/produit autre 429
Nombre de vente unique client/produit high-tech 23436


In [6]:
#On split le fichier de base en 2 parties : Train et Test
vente_train_autre, vente_test_autre = train_test_split(vente_autre, 
                                          test_size = 0.20)
print("Nombre de vente autre train %d" % len(vente_train_autre))
print("Nombre de vente autre test %d" % len(vente_test_autre))

vente_train_high_tech, vente_test_high_tech = train_test_split(vente_high_tech, 
                                          test_size = 0.20)
print("Nombre de vente high-tech train %d" % len(vente_train_high_tech))
print("Nombre de vente high-tech test %d" % len(vente_test_high_tech))

Nombre de vente autre train 343
Nombre de vente autre test 86
Nombre de vente high-tech train 18748
Nombre de vente high-tech test 4688


In [7]:
#On indexe pour l'id_client pour faciliter les recherches pendant l'évaluation du modèle
vente_full_indexed_autre = vente_autre.set_index('id_client')
vente_train_indexed_autre = vente_train_autre.set_index('id_client')
vente_test_indexed_autre = vente_test_autre.set_index('id_client')

vente_full_indexed_high_tech = vente_high_tech.set_index('id_client')
vente_train_indexed_high_tech = vente_train_high_tech.set_index('id_client')
vente_test_indexed_high_tech = vente_test_high_tech.set_index('id_client')

In [8]:
#Permet de recupérer les données clients et produit
def get_items_interacted(person_id, vente):
    interacted_items = vente.loc[person_id]['id_produit']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [9]:
#Nous avons choisit d'utiliser la métrique Recall@N
#Elle évalue si l'élément fait partie des N meilleurs recommendations
#La précision Top-N 
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

#CLASSE POUR LA CATEGORIE AUTRE
class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, vente_full_indexed_autre)
        all_items = set(vente_2015_autre['id_produit'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = vente_test_indexed_autre.loc[person_id]
        if type(interacted_values_testset['id_produit']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['id_produit'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['id_produit'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    vente_train_indexed_autre), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['id_produit'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['id_produit'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(vente_test_indexed_autre.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator_autre = ModelEvaluator()   

In [10]:
#Nous avons choisit d'utiliser la métrique Recall@N
#Elle évalue si l'élément fait partie des N meilleurs recommendations
#La précision Top-N 
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

#CLASSE POUR LA CATEGORIE HIGH-TECH
class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, vente_full_indexed_high_tech)
        all_items = set(vente_2015_high_tech['id_produit'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = vente_test_indexed_high_tech.loc[person_id]
        if type(interacted_values_testset['id_produit']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['id_produit'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['id_produit'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    vente_train_indexed_high_tech), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['id_produit'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['id_produit'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(vente_test_indexed_high_tech.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator_high_tech = ModelEvaluator()  

In [11]:
#Création de la matrice des clients / Produits avec 0 quand ils ont pas acheté le produit, et 1 quand ils ont acheté
#Matrice catégorie autre
matrix_vente_autre = vente_train_autre.pivot(index = 'id_client',
                                                 columns = 'id_produit',
                                                 values = 'note').fillna(0)

#Matrice catégorie High-tech 
matrix_vente_high_tech = vente_train_high_tech.pivot(index = 'id_client',
                                                 columns = 'id_produit',
                                                 values = 'note').fillna(0)


In [12]:
matrix_vente_autre_valeur = matrix_vente_autre.values
matrix_vente_high_tech_valeur = matrix_vente_high_tech.values

In [13]:
matrix_user_autre = list(matrix_vente_autre.index)
matrix_user_high_tech = list(matrix_vente_high_tech.index)

In [14]:
user_sparse_matrix_autre = csr_matrix(matrix_vente_autre_valeur)
user_sparse_matrix_autre

<49x275 sparse matrix of type '<class 'numpy.float64'>'
	with 343 stored elements in Compressed Sparse Row format>

In [15]:
user_sparse_matrix_high_tech = csr_matrix(matrix_vente_high_tech_valeur)
user_sparse_matrix_high_tech

<2978x15281 sparse matrix of type '<class 'numpy.float64'>'
	with 18748 stored elements in Compressed Sparse Row format>

In [16]:
nb_facteurs = 15
#Facteur pour le calcul sur les matrices
U_autre, sigma_autre, Vt_autre = svds(user_sparse_matrix_autre, k = nb_facteurs)
U_high_tech, sigma_high_tech, Vt_high_tech = svds(user_sparse_matrix_high_tech, k = nb_facteurs)

In [17]:
U_autre.shape
U_high_tech.shape

(2978, 15)

In [18]:
Vt_autre.shape
Vt_high_tech.shape

(15, 15281)

In [19]:
sigma_autre = np.diag(sigma_autre)
sigma_autre.shape

sigma_high_tech = np.diag(sigma_high_tech)
sigma_high_tech.shape

(15, 15)

In [20]:
#Nous allons calculer des prédicteurs pour les valeurs 0 de la matrice "matrix_vente_xx" 
user_predicted_autre = np.dot(np.dot(U_autre,sigma_autre), Vt_autre)

user_predicted_high_tech = np.dot(np.dot(U_high_tech, sigma_high_tech), Vt_high_tech)
user_predicted_high_tech

array([[ 1.80776797e-05,  1.66357929e-07,  4.68607570e-11, ...,
         2.84375191e-09,  5.52663682e-08,  4.98818208e-08],
       [ 7.31937503e-06,  5.75807214e-08,  1.79952178e-11, ...,
         3.68479984e-09,  1.06170839e-07,  9.81816457e-08],
       [ 2.49624826e-07,  1.84028271e-09,  1.80758921e-10, ...,
         2.14381181e-08,  2.48491868e-08,  1.17195807e-08],
       ...,
       [ 2.55063183e-07, -1.50719087e-08, -2.36210975e-11, ...,
         3.67225066e-09,  4.74511312e-06,  4.43540238e-06],
       [ 7.19767403e-06,  5.81059085e-09,  5.29980305e-12, ...,
         2.48205826e-10,  7.98710353e-09,  7.44603673e-09],
       [ 7.29181401e-07, -4.33154804e-08, -7.78018417e-11, ...,
         1.21303481e-08,  1.57536484e-05,  1.47254035e-05]])

In [21]:
user_predicted_autre_normalise = (user_predicted_autre - user_predicted_autre.min()) / (user_predicted_autre.max() - user_predicted_autre.min())

user_predicted_high_tech_normalise = (user_predicted_high_tech - user_predicted_high_tech.min()) / (user_predicted_high_tech.max() - user_predicted_high_tech.min())
user_predicted_high_tech_normalise

array([[0.03983242, 0.03981916, 0.03981904, ..., 0.03981904, 0.03981908,
        0.03981907],
       [0.03982446, 0.03981908, 0.03981904, ..., 0.03981904, 0.03981911,
        0.03981911],
       [0.03981922, 0.03981904, 0.03981904, ..., 0.03981905, 0.03981905,
        0.03981904],
       ...,
       [0.03981922, 0.03981902, 0.03981904, ..., 0.03981904, 0.03982255,
        0.03982232],
       [0.03982437, 0.03981904, 0.03981904, ..., 0.03981904, 0.03981904,
        0.03981904],
       [0.03981958, 0.039819  , 0.03981904, ..., 0.03981904, 0.0398307 ,
        0.03982994]])

In [22]:
matrix_preds_autre = pd.DataFrame(user_predicted_autre_normalise, columns = matrix_vente_autre.columns, index = matrix_user_autre).transpose()
matrix_preds_autre.head(10)

matrix_preds_high_tech = pd.DataFrame(user_predicted_high_tech_normalise, columns = matrix_vente_high_tech.columns, index = matrix_user_high_tech).transpose()
matrix_preds_high_tech.head(10)

,6,9,27,34,43,46,47,50,51,58,...,94690,94844,94898,95072,95673,95734,96875,98311,98610,98677
id_produit,,,,,,,,,,,,,,,,,,,,,
1,0.039832,0.039824,0.039819,0.040338,0.039819,0.039889,0.039820,0.039824,0.039819,0.039819,...,0.039819,0.039819,0.039819,0.039819,0.039816,0.039850,0.039822,0.039819,0.039824,0.039820
3,0.039819,0.039819,0.039819,0.039820,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,...,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819
4,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,...,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819
5,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,...,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819
7,0.039819,0.039819,0.039819,0.039823,0.039819,0.039820,0.039819,0.039819,0.039819,0.039819,...,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819
8,0.039820,0.039819,0.039819,0.039822,0.039819,0.039819,0.039819,0.039827,0.039819,0.039819,...,0.039819,0.039819,0.039819,0.039819,0.039816,0.039822,0.039819,0.039818,0.039819,0.039817
9,0.039819,0.039819,0.039819,0.039819,0.039819,0.039820,0.039819,0.039819,0.039819,0.039819,...,0.039819,0.039819,0.039819,0.039819,0.039827,0.039819,0.039819,0.039819,0.039819,0.039820
11,0.039819,0.039819,0.039819,0.039820,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,...,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819,0.039819
14,0.039821,0.039820,0.039819,0.039882,0.039819,0.039828,0.039819,0.039820,0.039819,0.039819,...,0.039819,0.039819,0.039819,0.039819,0.039819,0.039823,0.039819,0.039819,0.039820,0.039821


In [23]:
len(matrix_preds_autre.columns)
len(matrix_preds_high_tech.columns)

2978

In [24]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, matrix_preds_autre, items_df=None):
        self.cf_preds = matrix_preds_autre
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_preds[user_id].sort_values(ascending=False).reset_index().rename(columns={user_id: 'note'})

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['id_produit'].isin(items_to_ignore)].sort_values('note', ascending = False).head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df


        return recommendations_df
    
#Pour la catégorie Autre 
cf_recommender_model_autre = CFRecommender(matrix_preds_autre, vente_2015)

#Pour la catégorie High_tech 
cf_recommender_model_high_tech = CFRecommender(matrix_preds_high_tech, vente_2015)

In [25]:
print("SVD")
cf_global_metrics, cf_detailed_results = model_evaluator_autre.evaluate_model(cf_recommender_model_autre)
print("\nGlobal Metrics : \n%s" % cf_global_metrics)
cf_detailed_results.head(10)

SVD
40 users processed

Global Metrics : 
{'modelName': 'Collaborative Filtering', 'recall@5': 0.023255813953488372, 'recall@10': 0.03488372093023256}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
2,0,0,5,0.0,0.00,92611
0,0,0,4,0.0,0.00,96649
29,0,0,4,0.0,0.00,98430
14,0,1,4,0.0,0.25,98002
10,0,0,3,0.0,0.00,97733
24,0,0,3,0.0,0.00,98731
22,0,0,3,0.0,0.00,97663
21,0,0,3,0.0,0.00,98715
19,0,0,3,0.0,0.00,98147
20,0,0,3,0.0,0.00,97528


In [26]:
cf_recommender_model_autre.recommend_items(98704, topn=20, verbose=True)

,id_produit,note
0,99987,0.874124
1,99979,0.872147
2,99963,0.860688
3,99935,0.858675
4,99923,0.835527
5,99951,0.811632
6,99924,0.808621
11,51083,0.806517
14,95698,0.806517
13,99926,0.806517


In [27]:
vente_2015.loc[(vente_2015['id_client'] == 98704) & (vente_2015['id_produit'] == 99949)]

,id_vente,date_vente,id_client,categorie_client,id_magasin,prix_vente_unitaire,quantité,id_produit,categorie_produit,note
45310,6961,02/01/2015,98704,Autre,85,6.16,3,99949,Autre,1


In [28]:
vente_2015.loc[(vente_2015['id_client'] == 98704) & (vente_2015['id_produit'] == 99969)]

,id_vente,date_vente,id_client,categorie_client,id_magasin,prix_vente_unitaire,quantité,id_produit,categorie_produit,note


In [29]:
cf_recommender_model_high_tech.recommend_items(11317, topn=20, verbose=True)

,id_produit,note
0,11874,0.039819
2,4170,0.039819
3,1185,0.039819
4,14382,0.039819
1,9820,0.039819
10138,99192,0.039819
10143,99122,0.039819
10142,99130,0.039819
10141,99140,0.039819
10140,99167,0.039819


In [30]:
vente_2015.loc[(vente_2015['id_client'] == 11317) & (vente_2015['id_produit'] == 12240)]

,id_vente,date_vente,id_client,categorie_client,id_magasin,prix_vente_unitaire,quantité,id_produit,categorie_produit,note


In [31]:
#Calcul des produits "Autre" les plus populaires
item_popularity_autre = vente_autre.groupby('id_produit')['note'].sum().sort_values(ascending=False).reset_index()
item_popularity_autre.head(10)

item_popularity_high_tech = vente_high_tech.groupby('id_produit')['note'].sum().sort_values(ascending=False).reset_index()
item_popularity_high_tech.head(10)

,id_produit,note
0,1951,6.0
1,10793,6.0
2,5629,6.0
3,3589,6.0
4,10292,6.0
5,5436,6.0
6,1844,5.0
7,11993,5.0
8,3249,5.0
9,6167,5.0


In [32]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['id_produit'].isin(items_to_ignore)].sort_values('note', ascending = False).head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df

        return recommendations_df
    
#Produit Autre
popularity_model_autre = PopularityRecommender(item_popularity_autre, vente_2015)

#Produit High_tech
popularity_model_high_tech = PopularityRecommender(item_popularity_high_tech, vente_2015)

In [33]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = model_evaluator_autre.evaluate_model(popularity_model_autre)
print('\nGlobal metrics:\n%s' % pop_global_metrics)
pop_detailed_results_df.head(10)

Evaluating Popularity recommendation model...
40 users processed

Global metrics:
{'modelName': 'Popularity', 'recall@5': 0.06976744186046512, 'recall@10': 0.11627906976744186}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
2,0,0,5,0.000000,0.000000,92611
0,0,0,4,0.000000,0.000000,96649
29,0,0,4,0.000000,0.000000,98430
14,0,2,4,0.000000,0.500000,98002
10,0,0,3,0.000000,0.000000,97733
24,0,0,3,0.000000,0.000000,98731
22,1,1,3,0.333333,0.333333,97663
21,1,1,3,0.333333,0.333333,98715
19,0,1,3,0.000000,0.333333,98147
20,0,0,3,0.000000,0.000000,97528


In [34]:
popularity_model_autre.recommend_items(98704, topn=20, verbose=True)

,id_produit,note
0,99934,6.000000
1,99906,4.000000
2,99914,4.000000
3,99948,4.000000
4,99933,4.000000
5,99910,4.000000
6,99979,4.000000
7,99903,4.000000
9,99919,3.584963
8,99921,3.584963


In [35]:
vente_2015.loc[(vente_2015['id_client'] == 98704) & (vente_2015['id_produit'] == 99934)]

,id_vente,date_vente,id_client,categorie_client,id_magasin,prix_vente_unitaire,quantité,id_produit,categorie_produit,note


In [36]:
vente_2015.loc[(vente_2015['id_client'] == 98704) & (vente_2015['id_produit'] == 99919)]

,id_vente,date_vente,id_client,categorie_client,id_magasin,prix_vente_unitaire,quantité,id_produit,categorie_produit,note
45308,6961,02/01/2015,98704,Autre,85,20.83,8,99919,Autre,1
